In [2]:
import pandas as pd
from datetime import datetime
from pathlib import Path
import sqlite3 as sql

In [3]:
dtCarga = datetime.today().strftime('%d/%m/%Y %H:%M')

# COLETA DE DADOS DO SITE DO IBGE

In [4]:
#COLETANDO DADOS DO IBGE
url = 'https://www.ibge.gov.br/explica/codigos-dos-municipios.php#RJ'

DadosIBGE = pd.DataFrame(pd.read_html(url, match="Municípios do Rio de Janeiro")[0])

#RENOMEAR COLUNAS DO DF
DadosIBGE = DadosIBGE.rename(columns={'Municípios do Rio de Janeiro':'nmMunic','Códigos':'codMunic'})

#ALTERANDO NOME DO INDEX
DadosIBGE.index.name = 'idMunic'

#ALTERANDO INDEX PRA COMEÇAR EM 1
DadosIBGE.index = DadosIBGE.index + 1

#ADICIONAR UM COLUNA DE DATA E HORA DE CARGA
DadosIBGE['dtCarga'] = dtCarga

#DadosIBGE

# CRIANDO O BANCO DE DADOS ODS

In [5]:
#Manipulando o sistema de arquivos
endereco = Path('C:\\Users\\claud\\OneDrive\\Claudio Bonel-DADOTECA\\Projetos Sociais\\Projeto Dado Humanizado\\Pascoa com Dados\\')

BDODS = endereco / "pascoaODS.db"
BDDW = endereco / "pascoaDW.db"

if endereco.exists():
    if (BDODS.exists() and BDDW.exists()):
        print('Bancos de dados já existem!')
    else:
        BDODS.touch()
        BDDW.touch()
        print('Bancos de dados criados!')
else:
    print('Endereço não existe! Favor, verificar!')
        

Bancos de dados já existem!


# MANIPULANDO OS BANCOS DE DADOS CRIADOS

In [6]:
#Conectar no BDODS
conexaoODS = sql.connect(BDODS)

#Criar a tabela tbLogMunic e carregar os dados do DF DadosIBGE
DadosIBGE.to_sql('tbLogMunic',conexaoODS,if_exists="append")

#Confirmar a transação
conexaoODS.commit()

#Fechar a conexão
conexaoODS.close()

print('Carga do BDODS concluída!')

Carga do BDODS concluída!


In [7]:
#Conectar no BDDW
conexaoDW = sql.connect(BDDW)

#Selecionar somente as colunas para criação da dMunicipio
DadosIBGE = DadosIBGE[['nmMunic','codMunic']]

#Criar a tabela dMunicipio e carregar os dados do DF DadosIBGE
DadosIBGE.to_sql('dMunicipio',conexaoDW,if_exists="replace")

#Confirmar a transação
conexaoDW.commit()

#Fechar a conexão
conexaoDW.close()

print('Carga do DW concluída!')

Carga do DW concluída!


# CARGA DOS DADOS DE CADASTRO DAS DELEGACIAS DE POLÍCIA

In [8]:
#Definir o endereço físico do arquivo CSV
LocalArquivo = "C:\\Users\\claud\\OneDrive\\Claudio Bonel-DADOTECA\\CEPERJ\\Aulas\\Dados\\DADOS_POS\\"

#Criar Data Frame com os dados das DPs
tbLogDP = pd.DataFrame(pd.read_csv(LocalArquivo + "DP.csv"))

#Criando coluna com a data da carga
tbLogDP['dtCarga'] = dtCarga

#selecionando a coluna COD_DP como novo index
#tbLogDP = tbLogDP.set_index("COD_DP")

# CARGA DOS DADOS DE CADASTRO DOS RESPONSÁVEIS PELAS DELEGACIAS DE POLÍCIA

In [9]:
#Criar Data Frame com os dados dos responsáveis pelas DPs
tbLogRespDP = pd.DataFrame(pd.read_csv(LocalArquivo + "ResponsavelDP.csv"))

#Criando coluna com a data da carga
tbLogRespDP['dtCarga'] = dtCarga

#selecionando a coluna COD_DP como novo index
#tbLogRespDP = tbLogRespDP.set_index("COD_DP")

# CONECTANDO NO ODS E INFORMANDO AO PYTHON QUE USAREI LINGUAGEM SQL

In [10]:
#Conectar no BDODS
conexaoODS = sql.connect(BDODS)

#Definição de manipulação de dados em SQL
SQL_ODS = conexaoODS.cursor()

# INSERINDO DADOS DAS DELEGACIAS DE POLÍCIA, NO SQLITE (ODS) 

In [11]:
#Inserindo registros na tabela tbLogDP
print("Iniciando a inserção de dados na tabela tbLogDP.")
print("-------------------------------------------------")

SQL_ODS.executemany('''INSERT INTO tbLogDP (codDP,nmDP,endereco,dtCarga) VALUES (?,?,?,?)''',tbLogDP.values.tolist())

#confirmar a transação
conexaoODS.commit()

#Exibir uma mensagem de conclusão de carga
print("Carga finalizada!",len(tbLogDP), "registros inseridos na tbLogDP!")

Iniciando a inserção de dados na tabela tbLogDP.
-------------------------------------------------
Carga finalizada! 137 registros inseridos na tbLogDP!


# INSERINDO DADOS DOS RESPONSÁVEIS PELAS DELEGACIAS DE POLÍCIA, NO SQLITE (ODS) 

In [12]:
#Inserindo registros na tabela tbLogDP
print("Iniciando a inserção de dados na tabela tbLogRespDP.")
print("-------------------------------------------------")

SQL_ODS.executemany('''INSERT INTO tbLogRespDP (codDP,nmResponsavel,dtCarga) VALUES (?,?,?)''',tbLogRespDP.values.tolist())

#confirmar a transação
conexaoODS.commit()

#Exibir uma mensagem de conclusão de carga
print("Carga finalizada!",len(tbLogRespDP), "registros inseridos na tbLogDP!")

Iniciando a inserção de dados na tabela tbLogRespDP.
-------------------------------------------------
Carga finalizada! 137 registros inseridos na tbLogDP!


# VALIDANDO CARGA DOS DADOS DE DP E DE RESPONSÁVEL DP

In [13]:
#validar a carga através de um select na tblogDP
#pd.read_sql("SELECT * FROM tbLogDP",conexaoODS)

In [14]:
#validar a carga através de um select na tblogRespDP
#pd.read_sql("SELECT * FROM tbLogRespDP",conexaoODS)

# CRIANDO O DATAFRAME COM O SELECT DE CRIAÇÃO DA DIMENSÃO DP (dDP), NO DW

In [15]:
#Definir a query de criação da dimensão dDP

qry_dDP = '''
    SELECT 
        codDP,
        nmDP,
        endereco,
        nmResponsavel
    FROM
    (
        SELECT
            a.codDP,
            a.nmDP,
            a.endereco,
            b.nmResponsavel,
            max(a.dtCarga)
        FROM tbLogDP a
        JOIN tbLogRespDP b
        on a.codDP = b.codDP
        GROUP BY
            a.codDP,
            a.nmDP,
            a.endereco,
            b.nmResponsavel
    ) a
'''

dDP = pd.read_sql(qry_dDP,conexaoODS)

# FECHANDO A CONEXÃO COM O ODS

In [16]:
#Confirmar a transação
conexaoODS.commit()

#Fechar a conexão
conexaoODS.close()

# CONECTANDO NO DW E INFORMANDO AO PYTHON QUE USAREI LINGUAGEM SQL

In [29]:
#Conectar no BDODS
conexaoDW = sql.connect(BDDW)

#Definição de manipulação de dados em SQL
SQL_DW = conexaoDW.cursor()

# DELETANDO DADOS ANTIGOS E REINICIANDO AO AUTOINCREMENTAL DA CHAVE

In [30]:
#Deletar os dados atuais
SQL_DW.execute("DELETE FROM dDP")

#Reiniciando o autoincramental da chave
SQL_DW.execute("UPDATE sqlite_sequence SET seq=0 WHERE name='dDP'")

#Confirmar a transação
conexaoDW.commit()

# INSERINDO DADOS NA DIMENSÃO DP (dDP), NO DW

In [31]:
#Inserindo registros na tabela dDP
print("Iniciando a inserção de dados na tabela dDP.")
print("-------------------------------------------------")

SQL_DW.executemany('''INSERT INTO dDP (codDP,nmDP,endereco,nmResponsavel) VALUES (?,?,?,?)''',dDP.values.tolist())

#confirmar a transação
conexaoDW.commit()

#Exibir uma mensagem de conclusão de carga
print("Carga finalizada!",len(dDP), "registros inseridos na tbLogDP!")

Iniciando a inserção de dados na tabela dDP.
-------------------------------------------------
Carga finalizada! 137 registros inseridos na tbLogDP!


# VALIDANDO CARGA DOS DADOS DA DIMENSÃO DP (dDP)

In [32]:
#validar a carga através de um select na tblogDP
pd.read_sql("SELECT * FROM dDP",conexaoDW)

,idDP,codDP,nmDP,endereco,nmResponsavel
0,1,1,001ª DP - Praça Mauá,Atendendo provisoriamente na sede da 4ª DP - P...,Delegado de Polícia José Luiz Silva Duarte
1,2,4,004ª DP - Praça da República,"Av. Presidente Vargas, 1100 - Centro, Rio de J...",Delegada de Polícia Patricia de Paiva Aguiar
2,3,5,005ª DP - Mem de Sá,"Avenida Gomes Freire, 320 - Centro, Rio de Jan...",Delegado de Polícia Bruno Gilaberte Freitas
3,4,6,006ª DP - Cidade Nova,"Rua Professor Clementino Fraga, 77 - Centro, R...",Delegado de Polícia Fabio Luiz Da Silva Souza
4,5,7,007ª DP - Santa Teresa,"Rua Francisco de Castro, 5 - Santa Teresa, Rio...",Delegado de Polícia Carlos Alberto Meirelles D...
...,...,...,...,...,...
132,133,159,159ª DP - Cachoeira de Macacu,"Avenida Lord Baden Powel, 93 - Centro, Cachoei...",Delegado de Polícia Clovis Souza Moreira
133,134,165,165ª DP - Mangaratiba,"Estrada São João Marcos, s/n - Praia do Saco, ...",Delegado de Polícia Anderson Ribeiro Pinto
134,135,166,166ª DP - Angra dos Reis,"Rua Doutor Coutinho, 6 - Centro, Angra dos Rei...",Delegado de Polícia Vilson de Almeida Silva
135,136,167,167ª DP - Paraty,"Rua Roberto da Silveira , s/n - Vila Colonial,...",Delegado de Polícia Marcelo dos Santos Haddad


# FECHANDO A CONEXÃO COM O DW

In [28]:
#Confirmar a transação
conexaoDW.commit()

#Fechar a conexão
conexaoDW.close()